# Clase Práctica 2: Búsqueda Adversarial
---

Definimos la clase abstracta `Game`, para juegos de turnos de *n* jugadores. Se basa en el concepto de un "estado"  del juego `state`. Por ahora, todo lo que necesitamos es que un estado tenga un atributo `state.to_move`, que da el nombre del jugador del cual es el turno. ("Nombre" será algo así como `'X'` o `'O'` para tic-tac-toe.)

También definimos `play_game`, que toma un juego y un diccionario de pares `{jugador:strategia}`, y juega el juego, en cada turno comprobando `state.to_move` para ver de quién es el turno, y luego obteniendo la función de estrategia para ese jugador y aplicarla al juego y al estado para obtener un movimiento.

In [4]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError
        

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move)
            print(state)
    return state

# Ejercicio: Implementar Minimax

In [8]:
from email.errors import NonPrintableDefect
from algorithms import *

def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""
    player = state.to_move()
    return max_value(game , player ,state)


def max_value (game, player ,state):
    if game.is_terminal(state):
            return game.utility(state, player), None
    v , move = -infinity , None
    for i in game.action(state):
        v1 , _ = min_value(game , player , game.result(state,i))
        if(v1 > v):
            v , move = v1 , i
    return v , move


def min_value(game, player , state ):
    if game.is_terminal(state):
            return game.utility(state, player), None
    v , move = +infinity , None
    for i in game.action(state):
        v1 , _ = max_value(game , player , game.result(state,i))
        if(v1 < v):
            v , move = v1 , i
    return v , move
    

# Ejercicio: Añadir poda Alpha-Beta

In [10]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""
    player = state.to_move()
    return max_value(game , player ,state, -infinity , +infinity)


def max_value (game, player ,state , alpha , beta ):
    if game.is_terminal(state):
            return game.utility(state, player), None
    v , move = -infinity , None
    for i in game.action(state):
        v1 , _ = min_value(game , player , game.result(state,i),alpha , beta)
        if(v1 > v):
            v , move = v1 , i
            alpha = max(v,alpha)
        if(alpha >= beta):
            return v , move
    return v , move


def min_value(game, player , state , alpha , beta):
    if game.is_terminal(state):
            return game.utility(state, player), None
    v , move = +infinity , None
    for i in game.action(state):
        v1 , _ = max_value(game , player , game.result(state,i),alpha , beta)
        if(v1 < v):
            v , move = v1 , i
            alpha = max(v,alpha)
        if(alpha <= beta):
            return v , move
    return v , move

# Tic-Tac-Toe

Tenemos la noción de un juego abstracto y vamos a definir un juego real: Tic-tac-toe. Los movimientos son pares `(x, y)` que denotan cuadrados, donde `(0, 0)` es la parte superior izquierda y `(2, 2)` es la parte inferior derecha (en un tablero de tamaño `height=width=3 `).

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)     


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

Los estados en tic-tac-toe (y otros juegos) se representarán como un `Board`, que es una subclase de `defaultdict` que, en general, consistirá en pares `{(x, y): contents}`, por ejemplo `{(0, 0): 'X', (1, 1): 'O'}` podría ser el estado del tablero después de dos movimientos. Además del contenido de los cuadrados, un tablero también tiene algunos atributos:
- `.to_move` para nombrar el jugador cuyo movimiento es;
- `.width` y `.height` para dar el tamaño del tablero 
- posiblemente otros atributos, según lo especificado por las palabras clave.

Como `defaultdict`, la clase `Board` tiene un método `__missing__`, que devuelve `empty` para los cuadrados que no han sido asignados pero que están dentro de los límites de `width` × `height`, o `off` de lo contrario.

In [11]:
from collections import defaultdict
class Board(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

# Jugadores

Como interfaz para los jugadores, se representará a un jugador como un `callable` al que se le pasarán dos argumentos: `(game, state)` y devolverá un movimiento.
La función `player` crea un jugador a partir de un algoritmo de búsqueda, pero es posible crear jugadores como funciones, como se hace con `random_player` a continuación:

In [12]:
import random

def random_player(game, state): return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

# Let's Play! :)

Veamos que pasa en una partida de Tic-Tac-Toe entre un `random_player` (que elige aleatoriamente entre los movimientos legales) y un `player(alphabeta_search)`(que hace el movimiento óptimo). 

In [13]:
play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility

NameError: name 'TicTacToe' is not defined

El `player(alphabeta_search)` nunca perderá, pero si `random_player` tiene suerte, será un empate

Ahora veamos qué pasa si enfrentamos los algoritmos de búsqueda `minimax_search` y `alphabeta_search`  

In [14]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility

NameError: name 'TicTacToe' is not defined

Cuando compiten dos jugadores óptimos (alfa-beta o minimax), siempre será un empate

# Hex

Hex es un juego de mesa de estrategia abstracta para dos jugadores en el que los jugadores intentan conectar los lados opuestos de un tablero en forma de rombo hecho de celdas hexagonales. Hex fue inventado por el matemático y poeta Piet Hein en 1942 y luego redescubierto y popularizado por John Nash.

Se juega tradicionalmente en un tablero de rombos de 11×11, aunque también son populares los tableros de 13×13 y 19×19. El tablero está compuesto por hexágonos llamados celdas o hexágonos. A cada jugador se le asigna un par de lados opuestos del tablero, que deben intentar conectar colocando alternativamente una piedra de su color en cualquier hexágono vacío. Una vez colocadas, las piedras nunca se mueven ni se quitan. Un jugador gana cuando conecta con éxito sus lados a través de una cadena de piedras adyacentes. Los empates son imposibles en Hex debido a la topología del tablero de juego

![](hex-board-2.png)

In [15]:
class Hex(Game):
    """ Blue should connect left side with right side.
        Red should connect upper side with lower side.
        Blue makes first move.
    """
    
    def __init__(self, size=11):
        self.squares = {(x, y) for x in range(size) for y in range(size)}
        self.initial = HexBoard(height=size, width=size, to_move='B', ds = [-1] * (size * size + 4), utility=0)
        self.size = size

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        ds = board.ds[:]
        board = board.new({square: player}, to_move=('R' if player == 'B' else 'B'), ds = ds)
        
        x, y = square
        if player == 'B':
            if y == 0:
                board.join(0, self.position(x, y))
            elif y + 1 == self.size:
                board.join(1, self.position(x, y))
        else:
            if x == 0:
                board.join(2, self.position(x, y))
            elif x + 1 == self.size:
                board.join(3, self.position(x, y))

        for nx, ny in self.neighbour(x, y):
            if board[(nx, ny)] == board[(x, y)]:
                board.join(self.position(nx, ny), self.position(x, y))
        
        board.utility = (0 if not self.completed_path(board) else +1 if player == 'B' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'B' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)
    
    def completed_path(self, board):
        return board.root(0) == board.root(1) or board.root(2) == board.root(3)
    
    def checkInside(self, x, y):
        return 0 <= x and x < self.size and 0 <= y and y < self.size

    def neighbour(self, x, y):
        neighborhood = [(-1, 1), (0, 1), (1, 0), (1, -1), (0, -1), (-1, 0)]

        for neig in neighborhood:
            nx, ny = x + neig[0], y + neig[1]
            if self.checkInside(nx, ny):
                yield nx, ny

    def position(self, x, y):
        return self.size * x + y + 4

In [16]:
class HexBoard(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'W' or 'B'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=8, height=8, to_move=None, ds = None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, ds = ds,**kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = HexBoard(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        ans = ""
        for i in range(self.width):
            ans += " " * i
            for j in range(self.height):
                ans += " %s"%self[i, j]
            ans += "\n"
        return ans
    
    def root(self, a):
        if self.ds[a] < 0:
            return a
        else:
            self.ds[a] = self.root(self.ds[a])
            return self.ds[a]

    def join(self, a, b):
        a, b = self.root(a), self.root(b)
        if a == b:
            return False
        self.ds[a] = b
        return True


In [17]:
play_game(Hex(3), dict(B=random_player, R=player(alphabeta_search)), verbose=True).utility

Player B move: (2, 0)
 . . .
  . . .
   B . .



TypeError: 'str' object is not callable

# Ejercicio: Limitar la profundidad en la búsqueda

Para hacer uso de nuestro tiempo de cálculo limitado, podemos cortar la búsqueda temprano y aplicar un
función de evaluación heurística a estados, tratando efectivamente los nodos no terminales como si lo fueran.

In [ ]:
def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def h_alphabeta_search(game, state, cutoff=cutoff_depth(6), h=lambda s, p: 0):
    #TODO: Your code here!
    return None

# Ejercicio: Proponer una heurística para Hex

In [ ]:
def hex_heuristic(state, player):
    # TODO: Your code here!
    return 0

def hex_alphabeta_search(game, state, cutoff=cutoff_depth(6), h=hex_heuristic):
    return h_alphabeta_search_solution(game, state, cutoff, h)

In [ ]:
play_game(Hex(3), dict(B=random_player, R=player(h_alphabeta_search)), verbose=True).utility